<a href="https://colab.research.google.com/github/shyamchoure/CollabProjects/blob/main/NewsDetection/news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df_true = pd.read_csv('/content/gdrive/MyDrive/dataset/DataSet_Misinfo_TRUE.csv')
df_true.reset_index(drop=True )

,Unnamed: 0,text
0,0,The head of a conservative Republican faction ...
1,1,Transgender people will be allowed for the fir...
2,2,The special counsel investigation of links bet...
3,3,Trump campaign adviser George Papadopoulos tol...
4,4,President Donald Trump called on the U.S. Post...
...,...,...
34970,34970,Most conservatives who oppose marriage equalit...
34971,34971,The freshman senator from Georgia quoted scrip...
34972,34972,The State Department told the Republican Natio...
34973,34973,"ADDIS ABABA, Ethiopia —President Obama convene..."


In [ ]:
df_true['label']=1
df_true = df_true.drop('Unnamed: 0',axis=1)
df_true.head()

,text,label
0,The head of a conservative Republican faction ...,1
1,Transgender people will be allowed for the fir...,1
2,The special counsel investigation of links bet...,1
3,Trump campaign adviser George Papadopoulos tol...,1
4,President Donald Trump called on the U.S. Post...,1


In [ ]:
df_false = pd.read_csv('/content/gdrive/MyDrive/dataset/DataSet_Misinfo_FAKE.csv')
df_false

,Unnamed: 0,text
0,0,Donald Trump just couldn t wish all Americans ...
1,1,House Intelligence Committee Chairman Devin Nu...
2,2,"On Friday, it was revealed that former Milwauk..."
3,3,"On Christmas day, Donald Trump announced that ..."
4,4,Pope Francis used his annual Christmas Day mes...
...,...,...
43637,44422,The USA wants to divide Syria.\r\n\r\nGreat Br...
43638,44423,The Ukrainian coup d'etat cost the US nothing ...
43639,44424,The European Parliament falsifies history by d...
43640,44425,The European Parliament falsifies history by d...


In [ ]:
df_false['label']=0
df_false = df_false.drop('Unnamed: 0', axis=1 )
df_false.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [ ]:
df_main = pd.concat([df_true , df_false])
df_main.dropna(how="any" ,inplace=True)

In [ ]:
df_main.shape

(78588, 2)

In [ ]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
english_stops = set(stopwords.words('english'))

In [ ]:
x_data = df_main['text']
y_data = df_main['label']

In [ ]:
x_data.head(3)

0    The head of a conservative Republican faction ...
1    Transgender people will be allowed for the fir...
2    The special counsel investigation of links bet...
Name: text, dtype: object

In [ ]:
y_data.head(3)

0    1
1    1
2    1
Name: label, dtype: int64

In [ ]:
x_data = x_data.replace({'<.*?>': ''}, regex = True)
x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True) 

In [ ]:
x_data

0        The head of a conservative Republican faction ...
1        Transgender people will be allowed for the fir...
2        The special counsel investigation of links bet...
3        Trump campaign adviser George Papadopoulos tol...
4        President Donald Trump called on the U S  Post...
                               ...                        
43637    The USA wants to divide Syria     Great Britai...
43638    The Ukrainian coup d etat cost the US nothing ...
43639    The European Parliament falsifies history by d...
43640    The European Parliament falsifies history by d...
43641    A leading FSB officer  Segey Beseda  said duri...
Name: text, Length: 78588, dtype: object

In [ ]:
x_data = x_data.apply(lambda text: [w for w in text.split() if w not in english_stops])

In [ ]:
x_data = x_data.apply(lambda text: [w.lower() for w in text])

In [ ]:
from sklearn.model_selection import train_test_split 

ImportError: ignored

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x_data , y_data , test_size = 0.3 , random_state = 12345)

In [ ]:
x_train.shape

(55011,)

In [ ]:
def get_max_length():
    text_length = []
    for text in x_train:
        text_length.append(len(text))
    return int(np.ceil(np.mean(text_length)))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
token = Tokenizer(lower =False)

In [ ]:
token.fit_on_texts(x_train)

In [ ]:
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

In [ ]:
max_length = get_max_length()

In [ ]:
max_length

288

In [ ]:
x_test = pad_sequences(x_test, 
                       maxlen=max_length, 
                       padding='post', 
                       truncating='post')

In [ ]:
x_train = pad_sequences(x_train, 
                       maxlen=max_length, 
                       padding='post', 
                       truncating='post')

In [ ]:
total_words = len(token.word_index) + 1   # add 1 because of 0 padding
print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[   33     3    43 ...   990   700     0]
 [  308   116   540 ...     0     0     0]
 [    1 33992    55 ...     0     0     0]
 ...
 [  432    21   269 ...     0     0     0]
 [    1   209   193 ... 28346   136   223]
 [    1   859 53227 ...     0     0     0]] 

Encoded X Test
 [[ 2288   264  8893 ...     0     0     0]
 [ 3554   108    56 ...   375   243   162]
 [ 4527  5759   488 ...     0     0     0]
 ...
 [   16   194   460 ...     0     0     0]
 [    7    33     3 ...  3180  1284   735]
 [  833  6064   130 ... 15990   689  7057]] 

Maximum review length:  288


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , LSTM , Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(total_words,
                    32 , 
                     input_length = max_length))

In [ ]:
model.add(LSTM(256, 
               return_sequences=True))
model.add(LSTM(256,return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 288, 32)           5502368   
                                                                 
 lstm (LSTM)                 (None, 288, 256)          295936    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 6,323,873
Trainable params: 6,323,873
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train, validation_split=0.3, epochs=6)

Epoch 1/6
1204/1204 [==============================] - 3545s 3s/step - loss: 0.6667 - accuracy: 0.5791 - val_loss: 0.6818 - val_accuracy: 0.5537
Epoch 2/6
  20/1204 [..............................] - ETA: 51:53 - loss: 0.6640 - accuracy: 0.6016

In [ ]:
y_pred = (model.predict(x_test) >= 0.5).astype("int")

In [ ]:
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
accuracy_score(y_test, y_pred)


0.9463884294015354

In [ ]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.97      0.95     13153
           1       0.96      0.92      0.94     10424

    accuracy                           0.95     23577
   macro avg       0.95      0.94      0.95     23577
weighted avg       0.95      0.95      0.95     23577

